In [19]:
# pip install torch torchvision

In [20]:
# pip install cython

In [7]:
pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

Looking in links: https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [8]:
git clone https://github.com/facebookresearch/detectron2.git

SyntaxError: invalid syntax (782462151.py, line 1)

In [2]:
import torch
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import requests

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [3]:
from detectron2 import model_zoo
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

ModuleNotFoundError: No module named 'detectron2'

In [4]:
# Load the CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "openai/clip-vit-base-patch16"
processor = CLIPProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name).to(device)

In [10]:
# Load the Detectron2 object detection model
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # Adjust this threshold as needed
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_R_101_FPN_3x.yaml")
predictor = DefaultPredictor(cfg)

NameError: name 'get_cfg' is not defined

In [5]:
# Function to find the object in an image
def find_object_in_image(object_name, image_path):
    # Encode the object name and image
    text = object_name
    image = Image.open(requests.get(image_path, stream=True).raw)
    inputs = processor(text, images=image, return_tensors="pt").to(device)

    # Get the image and text embeddings
    with torch.no_grad():
        image_features = model.get_image_features(inputs.pixel_values).to(device)
        text_features = model.get_text_features(inputs.input_ids).to(device)

    # Compute the similarity score between image and text features
    similarity = (image_features @ text_features.T).squeeze(0)

#     # Use object detection to find the object's location
#     image_array = np.array(image)
#     outputs = predictor(image_array)

#     instances = outputs["instances"]
#     object_boxes = instances.pred_boxes.tensor.cpu().numpy()
#     object_scores = instances.scores.cpu().numpy()
    
#     # Find the index with the highest score
#     best_index = np.argmax(object_scores)

#     # Display the location of the object
#     x1, y1, x2, y2 = object_boxes[best_index]
#     image_with_box = cv2.rectangle(image_array, (int(x1), int(y1)), (int(x2), int(y2), (0, 0, 255), 2))

#     # Show the image with the bounding box
#     plt.imshow(cv2.cvtColor(image_with_box, cv2.COLOR_BGR2RGB))
#     plt.title(f"Object Location: {object_name}")
#     plt.show()
    
    return similarity.item()

In [6]:
# Example usage
object_name = "chair"
image_url = "https://img.homejournal.com/202005/5eb28b30b7c7c.jpg"
similarity_score = find_object_in_image(object_name, image_url)
print(f"Similarity score for '{object_name}' in the image: {similarity_score:.2f}")

Similarity score for 'chair' in the image: 23.61
